In [2]:
import os
import numpy as np
import pandas as pd
import json

In [7]:
results_path = os.path.join("..","trained_models")
experiments_name = [folder for folder in os.listdir(results_path) 
                    if os.path.isdir(os.path.join(results_path, folder))]

experiments_dict = {
    'datetime':[],
    'dataset':[],
    'model':[],
    'distribution_head':[],
    'scaling':[],
    'model_params':[],
    'epochs':[],
    'early_stop':[],
    'actual':[],
    'forecast':[],
    'metrics':[]
}
for exp in experiments_name:
    exp_split = exp.split('__')
    experiments_dict['dataset'].append( exp_split[1] )
    experiments_dict['model'].append( exp_split[0] )
    experiments_dict['datetime'].append( exp_split[2] )
    experiments_dict['actual'].append( os.path.join(results_path,exp,'actuals.npy') )
    experiments_dict['forecast'].append( os.path.join(results_path,exp,'forecasts.npy') )
    exp_info = json.load(open(os.path.join(results_path,exp,'experiment.json'), "r"))
    experiments_dict['distribution_head'].append( exp_info['distribution_head'] )
    experiments_dict['scaling'].append( exp_info['scaling'] if exp_info['scaling'] else '-' )
    experiments_dict['epochs'].append( exp_info['epoch'] )
    experiments_dict['early_stop'].append( exp_info['early_stop'] )
    experiments_dict['metrics'].append( json.load(open(os.path.join(results_path,exp,'metrics.json'), "r")) )
    experiments_dict['model_params'].append( json.load(open(os.path.join(results_path,exp,'model_params.json'), "r")) )

In [8]:
experiments_df = pd.DataFrame(experiments_dict)
experiments_df['datetime'] = pd.to_datetime(experiments_df['datetime'], format="%Y-%m-%d-%H:%M:%S:%f")
experiments_df = experiments_df[experiments_df.dataset == "M5"]
experiments_df.set_index(['datetime','dataset','model','distribution_head','scaling'], inplace=True)

quantile_losses = pd.DataFrame(pd.DataFrame(values['metrics']['quantile_loss']).mean(axis=0)
                               for _, values in experiments_df.iterrows())
quantile_losses.set_index(experiments_df.index, inplace=True)
quantile_losses.style.apply(lambda x: ['background-color: green' if v == x.min() else '' for v in x])

,,,,,q0.25,q0.5,q0.8,q0.9,q0.95,q0.99
datetime,dataset,model,distribution_head,scaling,,,,,,
2024-03-28 17:36:13.933005,M5,transformer,poisson,-,4471.382076,3994.472712,3422.181476,3231.417730,3136.035858,3059.730359
2024-03-28 15:09:32.178075,M5,deepAR,tweedie,-,3948.807288,4033.918393,4136.051720,4170.096162,4187.118383,4200.736160
2024-03-28 15:22:08.896495,M5,deepAR,poisson,-,3611.899147,3495.956576,3356.825490,3310.448462,3287.259948,3268.709136
2024-03-28 15:10:02.154659,M5,deepAR,negbin,-,3895.391637,3920.809380,3951.310672,3961.477770,3966.561318,3970.628157
2024-03-28 18:39:55.296393,M5,transformer,tweedie,-,3371.986199,3815.795610,4348.366902,4525.890667,4614.652549,4685.662054
2024-03-28 17:23:19.293995,M5,transformer,negbin,-,5041.209757,4409.477698,3651.399226,3398.706402,3272.359990,3171.282861
